# ASSET demonstration

This notebook aims at showing how to use the ASSET package to extract a spectrum
from log-slit spectroscopy data. This example extracts the spectrum of a star
companion from coronagraphic data.


In [ ]:
# Load packages

using EasyFITS
using InverseProblem
using LinearInterpolators
using PyPlot
const plt = PyPlot

using ASSET


In [ ]:
# Load data

#read fits files
dir = "<PATH TO DATA>"
data = readfits(joinpath(dir, "preprocessed_data.fits.gz"))
wgt = readfits(joinpath(dir, "preprocessed_data_weights.fits.gz"))
calib = readfits(joinpath(dir, "geometric_calibration.fits.gz"))
rho = calib[:, :, 1]
lambda = calib[:, :, 2]
mask = calib[:, :, 3]

# angular separation from companion to center of coronagraphic mask
rho_comp = -800*Exospeco.mas

# create a CalibratedData structure
D = CalibratedData(data, wgt, rho .- rho_comp, lambda)


In [ ]:
# Define the direct model and regularization parameters

#spectral interpolator
λmin=minimum(D.λ_map[D.λ_map.!=0])
λmax=maximum(D.λ_map[D.λ_map.!=0])
λz=range(λmin, length=Nz, stop=λmax)
ker = CatmullRomSpline(Float64, Flat)
F = SparseInterpolator(ker, D.λ_map, λz)

#psf model
psf_center_bnds = 1e2
αinit = 0.1*maximum(D.λ_map)^(-2)
h = chromGaussianPSF(αinit)
psf_params_bnds = [(0., 1e2)]

#companion SED regularization
hpz = 5e4
Rz = hpz*tikhonov()

#background regularization
Rb = 1e1*tikhonov()
b = BkgMdl(zeros(size(D)), Rb)


In [ ]:
# Extract the spectrum

#initialize
Nz = size(D)[2]
z = zeros(Nz)

z, hfp = extract_spectrum!(z, F, h, D ,Rz, b;
                           auto_calib=Val(false),
                           extract_kwds=(psf_params_bnds=psf_params_bnds, 
                                         psf_center_bnds=psf_center_bnds),
                           loss_tol=(0.0, 1e-5), z_tol=(0.0, 1e-5))
